<h1>Imports

In [ ]:
!pip install pip install setuptools


In [ ]:
import pandas as pd
from pyspark.sql import (SparkSession,
                        functions as F,
                        DataFrame,
                        Window)
import matplotlib as plt


<h3>Create spark session

In [ ]:

spark = SparkSession.builder \
    .appName("Analises") \
    .getOrCreate()
spark


<h2>Functions

In [14]:
def takePandas(self, n):
    rows = self.take(n)
    df = pd.DataFrame(rows, columns=self.columns)
    return df
DataFrame.takePandas = takePandas


<h1>Code

In [ ]:
df_lol = spark.read.csv("..\..\datasets\League of legend Champions 2024.csv",header=True)

In [ ]:
df_lol.takePandas(10)

In [ ]:
df_lol.printSchema()

<h1> Roles distribution

Pergunta: Quais papéis (Role) são mais comuns entre os campeões?<br>
Insight: Distribuição percentual dos campeões por papel (ex.: Lutador, Mago, Suporte).<br>
Benefício: Identificar qual categoria de campeões é mais dominante e se há algum desequilíbrio entre os papéis.

In [ ]:
df_lol.count()

In [ ]:
roles_distribution = (df_lol.groupBy("Classes")
                      .count()
                      .withColumn("share",F.col("count")/df_lol.count())
                      .orderBy(F.desc("count")))

roles_distribution.takePandas(3)

In [ ]:
roles_distribution.toPandas().plot()